In [43]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from os import listdir
import seaborn as sns
sns.set_style("white")

In [44]:
from keras.preprocessing import sequence
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from keras import optimizers
from keras.regularizers import l1,l2,l1_l2

from keras.optimizers import Adam
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json

In [45]:
df = pd.read_csv('df_test.csv')
df

,5160399000.0,5160532000.0,5160665000.0,5160798000.0,5160931000.0,5161064000.0,5161197000.0,5161330000.0,5161463000.0,5161596000.0,...,5178354000.0,5178487000.0,5178620000.0,5178753000.0,5178886000.0,5179019000.0,5179152000.0,5179285000.0,5179418000.0,target
0,2.950901,0.902424,3.609979,3.055115,4.036279,5.421765,5.709461,5.342877,5.316088,4.677352,...,4.535670,5.781560,5.968084,5.393829,4.983120,4.697924,2.387842,1.606726,0.182691,0
1,3.031936,3.011295,4.798773,5.328106,4.714895,6.324241,6.342548,4.747849,4.850721,4.482862,...,5.628841,6.765914,7.336272,7.453834,5.646930,4.853823,4.431760,4.068731,3.125760,0
2,6.530953,8.500089,12.586018,12.204521,10.663135,11.833899,14.070092,14.178874,12.044108,11.868625,...,11.027771,11.312739,13.174208,13.844385,11.373266,11.034321,10.539037,11.887417,10.584875,0
3,3.164263,5.529422,7.753607,6.913345,8.067882,8.296607,9.290557,7.724988,7.805755,7.959184,...,6.416645,5.814830,6.985933,8.193748,7.569565,8.249093,6.622108,5.638334,6.400860,0
4,8.386093,9.840102,12.134753,12.547019,12.631717,14.261529,13.987828,12.428510,10.985325,10.450608,...,12.186491,12.201777,14.507222,15.508496,13.201510,12.297471,11.698099,13.270782,11.070101,0
5,6.023662,11.058970,13.498687,10.400385,11.516341,13.994253,14.812970,13.592376,11.068720,10.359941,...,12.384987,11.421519,12.508307,13.805166,13.402535,12.459557,11.105112,11.641913,7.829498,0
6,11.855904,9.914749,11.850813,11.484727,12.554704,13.287239,12.812065,11.279834,9.964271,10.473292,...,11.888719,12.026691,14.322939,14.720829,13.583829,12.520845,10.796050,11.873114,10.827345,0
7,3.936545,8.210159,10.308178,9.670616,13.864596,14.127773,16.216343,15.077238,11.596483,12.203186,...,11.634393,11.355016,14.057089,15.128881,14.111677,14.175338,11.606353,12.873405,10.783709,0
8,7.094536,5.655632,8.893394,8.212312,10.392735,9.836021,10.882961,9.272468,8.711961,9.024563,...,9.418217,8.841332,10.676582,10.525266,10.156544,9.622389,7.725622,8.338539,7.338725,0
9,13.265439,10.537382,12.862325,10.062306,12.105781,15.105871,15.638883,13.266718,11.191602,10.314122,...,12.659743,10.854548,14.215042,15.054139,13.448993,12.928502,11.392657,12.206523,8.899485,0


In [46]:
unit_dict = {0 : 2,
             1 : 4,
             2 : 6,
             3 : 8,
             4 : 10}

def route_initializer(df, 
                      forward_order = [0, 3, 6, 9, 12, 15, 16, 13, 10, 7, 4, 1, 2, 5, 8, 11, 14, 17],
                      backward_order = [17, 14, 11, 8, 5, 2, 1, 4, 7, 10, 13, 16, 15, 12, 9, 6, 3, 0],
                      foward_size_of_units = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                      backward_size_of_units = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]):
   
    # split the dataframe and store each class as element in a list
    df_list = []
    for i in range(18):
        df_split = df[df['target'] == i]
        df_split.reset_index(drop = True, inplace = True)
        df_list.append(df_split)
        
    # specify the order of the each class based on route
    test_list_forward = [df_list[i] for i in forward_order]
    test_list_backward = [df_list[i] for i in backward_order]
    
    # pick random number of samples for each class based on size of units
    test_list_processed_forward = []
    test_list_processed_backward = []
    for i in range(18):
        test_list_processed_forward.append(test_list_forward[i].sample(n = unit_dict[foward_size_of_units[i]]))
        test_list_processed_backward.append(test_list_backward[i].sample(n = unit_dict[backward_size_of_units[i]]))
    
    return pd.concat(test_list_processed_forward), pd.concat(test_list_processed_backward)

In [47]:
df_test_forward, df_test_backward = route_initializer(df, 
            forward_order = [0, 3, 6, 9, 12, 15, 16, 13, 10, 7, 4, 1, 2, 5, 8, 11, 14, 17],
            backward_order = [17, 14, 11, 8, 5, 2, 1, 4, 7, 10, 13, 16, 15, 12, 9, 6, 3, 0],
            foward_size_of_units = [1, 1, 2, 2, 2, 4, 1, 1, 0, 2, 1, 1, 0, 0, 3, 0, 2, 3],                
            backward_size_of_units = [2, 1, 2, 1, 0, 1, 4, 0, 0, 1, 2, 0, 0, 1, 1, 0, 3, 3])
df_test = pd.concat([df_test_forward, df_test_backward])
df_test 

,5160399000.0,5160532000.0,5160665000.0,5160798000.0,5160931000.0,5161064000.0,5161197000.0,5161330000.0,5161463000.0,5161596000.0,...,5178354000.0,5178487000.0,5178620000.0,5178753000.0,5178886000.0,5179019000.0,5179152000.0,5179285000.0,5179418000.0,target
22,5.437886,8.606829,10.319074,7.407415,9.107817,9.447973,12.205886,11.717331,9.466329,10.069024,...,10.094877,10.088136,10.451743,11.556876,10.275420,9.263680,8.520674,9.991991,7.868459,0
29,1.982771,6.434322,9.824334,7.876110,9.100032,10.050710,10.997368,9.797427,8.069436,9.103599,...,8.809099,5.539208,9.288607,11.117620,9.616412,8.737240,6.154120,6.722514,6.544347,0
50,12.102742,12.236226,12.117069,13.247639,16.321514,15.517549,16.666277,16.291309,14.015028,14.793983,...,14.441759,14.747091,16.999958,17.262628,15.795294,17.472732,14.298231,16.262834,14.261239,0
62,4.578557,6.674959,8.352165,6.679683,8.267180,10.220278,10.674751,9.805678,8.269046,9.940350,...,9.219231,9.017539,8.548860,10.179949,9.343094,8.574915,7.848691,9.508656,8.449991,0
2,0.221053,-1.145537,-0.501613,2.326105,2.752305,2.358892,1.956454,2.773553,2.652963,2.772423,...,5.023408,6.540455,7.276174,7.507465,6.251981,5.367171,6.105089,3.981461,2.395100,3
74,-1.070548,0.686516,0.819000,0.306413,1.061842,2.037999,2.813986,2.656994,3.576440,4.112051,...,4.269926,4.577874,4.667971,4.968485,3.324227,2.017198,2.885843,2.109888,0.740938,3
53,-1.209950,-0.506903,1.136273,1.835914,2.488516,2.613028,3.250610,3.594146,3.365381,3.587451,...,4.914518,5.510143,5.733486,3.802937,3.869351,3.598449,2.132909,2.776192,0.683154,3
11,-0.711975,-0.003393,0.463791,1.211690,1.232702,2.065182,1.574770,2.815300,2.623954,3.571184,...,6.466936,5.460361,5.839198,6.535041,5.787418,4.890803,6.424007,4.893807,2.607269,3
67,1.116333,1.805307,4.838686,5.577200,6.265129,5.877710,5.317664,5.530762,4.281115,3.694697,...,4.702755,4.349603,4.867025,5.952008,6.294654,4.916939,3.223938,4.123229,4.311948,6
14,8.350067,9.895012,11.383849,12.272936,9.586447,10.769463,12.305288,12.419008,10.132106,9.844156,...,8.299681,10.149776,11.677543,11.740472,10.895147,11.798797,8.434296,9.480295,8.788835,6


In [48]:
df_med = df_test.drop(df_test.iloc[:, :36], axis = 1) 
df_med.drop(df_med.iloc[:, 72:108], inplace = True, axis = 1) 

df_small = df_test.drop(df_test.iloc[:, :54], axis = 1) 
df_small.drop(df_small.iloc[:, 36:90], inplace = True, axis = 1) 

df_smaller = df_test.drop(df_test.iloc[:, :64], axis = 1) 
df_smaller.drop(df_smaller.iloc[:, 18:80], inplace = True, axis = 1) 

In [49]:
df_smooth = df_test.T
df_med = df_med.T
df_small = df_small.T
df_smaller = df_smaller.T

In [50]:
sequences_smooth = list()
for i in range(df_smooth.shape[1]):
    values = df_smooth.iloc[:-1,i].values
    sequences_smooth.append(values)
targets_smooth = df_smooth.iloc[-1, :].values

sequences_med = list()
for i in range(df_med.shape[1]):
    values = df_med.iloc[:-1,i].values
    sequences_med.append(values)
targets_med = df_med.iloc[-1, :].values

sequences_small = list()
for i in range(df_small.shape[1]):
    values = df_small.iloc[:-1,i].values
    sequences_small.append(values)
targets_small = df_small.iloc[-1, :].values

sequences_smaller = list()
for i in range(df_smaller.shape[1]):
    values = df_smaller.iloc[:-1,i].values
    sequences_smaller.append(values)
targets_smaller = df_smaller.iloc[-1, :].values

targets = targets_smooth

In [51]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import np_utils

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(targets)
encoded_y = encoder.transform(targets)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_y)
targets = dummy_y

In [52]:
X_test_smooth, X_test_med, X_test_small, X_test_smaller, y_test = sequences_smooth, sequences_med, sequences_small, sequences_smaller, targets

In [53]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.externals.joblib import dump, load
sc1 = load('std_scaler_smooth.bin')
X_test_smooth = sc1.transform(X_test_smooth)

sc2 = load('std_scaler_med.bin')
X_test_med = sc2.transform(X_test_med)

sc3 = load('std_scaler_small.bin')
X_test_small = sc3.transform(X_test_small)

sc4 = load('std_scaler_smaller.bin')
X_test_smaller = sc4.transform(X_test_smaller)

In [54]:
X_test_smooth.shape, X_test_med.shape, X_test_small.shape, X_test_smaller.shape

((168, 144), (168, 72), (168, 36), (168, 18))

In [55]:
X_test_smooth = np.reshape(X_test_smooth, (X_test_smooth.shape[0], X_test_smooth.shape[1], 1))
X_test_med = np.reshape(X_test_med, (X_test_med.shape[0], X_test_med.shape[1], 1))
X_test_small = np.reshape(X_test_small, (X_test_small.shape[0], X_test_small.shape[1], 1))
X_test_smaller = np.reshape(X_test_smaller, (X_test_smaller.shape[0], X_test_smaller.shape[1], 1))

In [56]:
n_timesteps_smooth, n_features_smooth = X_test_smooth.shape[1], X_test_smooth.shape[2]
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps_smooth, n_length_smooth = 4, 36
X_test_smooth = X_test_smooth.reshape((X_test_smooth.shape[0], n_steps_smooth, 1, n_length_smooth, n_features_smooth))

n_timesteps_med, n_features_med = X_test_med.shape[1], X_test_med.shape[2]
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps_med, n_length_med = 3, 24
X_test_med = X_test_med.reshape((X_test_med.shape[0], n_steps_med, 1, n_length_med, n_features_med))

n_timesteps_small, n_features_small = X_test_small.shape[1], X_test_small.shape[2]
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps_small, n_length_small = 2, 18
X_test_small = X_test_small.reshape((X_test_small.shape[0], n_steps_small, 1, n_length_small, n_features_small))

n_timesteps_smaller, n_features_smaller = X_test_smaller.shape[1], X_test_smaller.shape[2]
# reshape into subsequences (samples, time steps, rows, cols, channels)
n_steps_smaller, n_length_smaller = 1, 18
X_test_smaller = X_test_smaller.reshape((X_test_smaller.shape[0], n_steps_smaller, 1, n_length_smaller, n_features_smaller))


In [57]:
X_test_smooth.shape

(168, 4, 1, 36, 1)

In [58]:
y_test.argmax(axis=1)

array([ 0,  0,  0,  0,  3,  3,  3,  3,  6,  6,  6,  6,  6,  6,  9,  9,  9,
        9,  9,  9, 12, 12, 12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 16, 16, 16, 16, 13, 13, 13, 13, 10, 10,  7,  7,  7,  7,  7,
        7,  4,  4,  4,  4,  1,  1,  1,  1,  2,  2,  5,  5,  8,  8,  8,  8,
        8,  8,  8,  8, 11, 11, 14, 14, 14, 14, 14, 14, 17, 17, 17, 17, 17,
       17, 17, 17, 17, 17, 17, 17, 17, 17, 14, 14, 14, 14, 11, 11, 11, 11,
       11, 11,  8,  8,  8,  8,  5,  5,  2,  2,  2,  2,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  4,  4,  7,  7, 10, 10, 10, 10, 13, 13, 13, 13,
       13, 13, 16, 16, 15, 15, 12, 12, 12, 12,  9,  9,  9,  9,  6,  6,  3,
        3,  3,  3,  3,  3,  3,  3,  0,  0,  0,  0,  0,  0,  0,  0])

In [59]:
class Model:
    def __init__(self, path_model, path_weight):
        self.model = self.loadmodel(path_model, path_weight)
        self.graph = tf.get_default_graph()    
    
    @staticmethod
    def loadmodel(path_model, path_weight):
        json_file = open(path_model, 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        model = model_from_json(loaded_model_json)
        model.load_weights(path_weight)
        return model

    def predict(self, X):
        with self.graph.as_default():
            return self.model.predict(X)

In [60]:
work_dir_model = '/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_models/'
work_dir_weight = '/home/hongyu/Documents/Spring2020/ECE_research/signal_analysis/data_18points/3_section_sliding5/MSLSTM_weights/'

# work_dir_model = '/home/wuh007/Desktop/signal/signal_analysis/data_18points/3_section/models_mixed/'
# work_dir_weight = '/home/wuh007/Desktop/signal/signal_analysis/data_18points/3_section/weights_mixed/'

In [61]:
model_convlstm = Model(work_dir_model + 'MSLSTM_18ptsconvlstm_model.json', work_dir_weight + 'model-030-0.984992-0.947412-0.172179-18ptsconvlstm.h5')
model_lstm_simple = Model(work_dir_model + 'lstm_simplemodels.json', work_dir_weight + 'model-256-0.060712-0.031487-simple.h5')
model_lstm_simple2 = Model(work_dir_model + 'lstm_simplemodels.json', work_dir_weight + 'model-237-0.067243-0.042658-simple2.h5')

In [62]:
y_pred_convlstm = model_convlstm.predict([X_test_small, X_test_med, X_test_smaller, X_test_smooth])

In [63]:
len(y_pred_convlstm), len(y_test)

(168, 168)

In [64]:
y_pred_convlstm

array([[9.9996269e-01, 4.5762019e-07, 8.3054426e-09, ..., 4.4355844e-22,
        8.7224725e-21, 4.5633606e-23],
       [3.0434835e-01, 8.3153918e-02, 2.2971838e-04, ..., 3.2233696e-16,
        5.2936126e-18, 4.3304992e-20],
       [3.7090516e-01, 2.5444249e-13, 1.4008679e-12, ..., 9.1696096e-28,
        5.5534107e-25, 5.9657058e-29],
       ...,
       [9.8472810e-01, 1.7242822e-04, 5.3139875e-04, ..., 1.2647574e-24,
        1.2941393e-22, 5.8370902e-28],
       [9.1031706e-01, 2.2763666e-04, 1.1970661e-04, ..., 5.1446131e-19,
        1.2507749e-16, 2.8047798e-20],
       [9.9763548e-01, 1.3587374e-03, 5.8333734e-07, ..., 1.1782533e-16,
        4.4629446e-16, 6.3137963e-22]], dtype=float32)

In [65]:
y_pred_convlstm.argmax(axis=1), len(y_pred_convlstm.argmax(axis=1))

(array([ 0,  6, 11,  0,  3,  3,  3,  3,  6,  6,  6,  6,  6,  6,  9,  9,  9,
         9,  9,  9, 12, 12, 12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 16, 16, 16, 16, 13, 13, 13, 13, 10, 10,  7,  7,  7,  7,  7,
         7,  4,  4,  4,  4,  1,  1,  1,  1,  2,  2,  5,  5,  8,  8,  8,  8,
         4, 10,  4,  8, 11, 11, 14, 14, 14, 14, 14, 14, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 14, 14, 14,  4, 11, 11,  3, 11,
        11, 11,  8,  8,  8,  8,  5,  5,  2,  0,  2,  2,  1,  1,  1,  1,  1,
         1,  1,  0,  1,  1,  4,  4,  7,  7, 10, 10, 10, 10, 13, 13, 13, 13,
        13, 17, 16, 16, 15, 15, 12, 12, 12, 12,  9,  9,  9,  9,  6,  6,  3,
         3,  3,  3,  3,  3, 11,  2, 11,  0,  0,  0,  6,  0,  0,  0]), 168)

In [66]:
y_pred_convlstm[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1), len(y_pred_convlstm[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1))

(array([ 0,  0,  3,  3,  3,  3,  6,  6,  6,  6,  6,  6,  9,  9,  9,  9,  9,
         9, 12, 12, 12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        16, 16, 16, 16, 13, 13, 13, 13, 10, 10,  7,  7,  7,  7,  7,  7,  4,
         4,  4,  4,  1,  1,  1,  1,  2,  2,  5,  5,  8,  8,  8,  8,  4,  4,
        11, 11, 14, 14, 14, 14, 14, 14, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 14, 14, 14,  4, 11,  3, 11, 11, 11,  8,  8,  8,
         5,  5,  2,  0,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  4,  4,
         7,  7, 10, 10, 10, 10, 13, 13, 13, 13, 13, 17, 16, 16, 15, 15, 12,
        12, 12, 12,  9,  9,  9,  9,  6,  6,  3,  3,  3,  3,  3,  3, 11,  0,
         0,  0,  0,  0,  0]), 158)

In [67]:
y_test[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1), len(y_test[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1))

(array([ 0,  0,  3,  3,  3,  3,  6,  6,  6,  6,  6,  6,  9,  9,  9,  9,  9,
         9, 12, 12, 12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15,
        16, 16, 16, 16, 13, 13, 13, 13, 10, 10,  7,  7,  7,  7,  7,  7,  4,
         4,  4,  4,  1,  1,  1,  1,  2,  2,  5,  5,  8,  8,  8,  8,  8,  8,
        11, 11, 14, 14, 14, 14, 14, 14, 17, 17, 17, 17, 17, 17, 17, 17, 17,
        17, 17, 17, 17, 17, 14, 14, 14, 14, 11, 11, 11, 11, 11,  8,  8,  8,
         5,  5,  2,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  4,  4,
         7,  7, 10, 10, 10, 10, 13, 13, 13, 13, 13, 13, 16, 16, 15, 15, 12,
        12, 12, 12,  9,  9,  9,  9,  6,  6,  3,  3,  3,  3,  3,  3,  3,  0,
         0,  0,  0,  0,  0]), 158)

In [68]:
sc_lstm = load('std_scaler_nextMove.bin')

In [70]:
# sliding window one step for all conditions

from collections import deque 
  
# sequential voting stategy
def sequential_voting(sequence = y_pred_convlstm[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1),
                      n_steps = 3):
    y_pred_new = list()
    i = 0
    for _ in range(len(sequence)):
        print(i, len(sequence))
        if i >= len(sequence)-2:
            break
        
        stack = deque([sequence[i], sequence[i+1],sequence[i+2]]) 
        print(stack)
        
        if len(set(stack)) == 1 or len(set(stack)) == 2:
            for item in set(stack):
                if stack.count(item) > 1:
                    y_pred_new.append(item) 
            i += 1
            
        else:
            test = sc_lstm.transform([stack])
            test = test.reshape(1, 3, 1)
            out, out2 = int(np.round(model_lstm_simple.predict(test))), int(np.round(model_lstm_simple2.predict(test)))
            if out == out2:
                y_pred_new.append(out)
            else:
                y_pred_new.append([out, out2])
            print([int(np.round(model_lstm_simple.predict(test))), 
                int(np.round(model_lstm_simple2.predict(test)))])
            i += 1
            
    
        print(y_pred_new)
    return y_pred_new
    
y_pred_new = sequential_voting()
# y_pred_new, len(y_pred_new)

0 158
deque([0, 0, 3])
[0]
1 158
deque([0, 3, 3])
[0, 3]
2 158
deque([3, 3, 3])
[0, 3, 3]
3 158
deque([3, 3, 3])
[0, 3, 3, 3]
4 158
deque([3, 3, 6])
[0, 3, 3, 3, 3]
5 158
deque([3, 6, 6])
[0, 3, 3, 3, 3, 6]
6 158
deque([6, 6, 6])
[0, 3, 3, 3, 3, 6, 6]
7 158
deque([6, 6, 6])
[0, 3, 3, 3, 3, 6, 6, 6]
8 158
deque([6, 6, 6])
[0, 3, 3, 3, 3, 6, 6, 6, 6]
9 158
deque([6, 6, 6])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6]
10 158
deque([6, 6, 9])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6]
11 158
deque([6, 9, 9])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 9]
12 158
deque([9, 9, 9])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 9, 9]
13 158
deque([9, 9, 9])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 9, 9, 9]
14 158
deque([9, 9, 9])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9]
15 158
deque([9, 9, 9])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9]
16 158
deque([9, 9, 12])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9]
17 158
deque([9, 12, 12])
[0, 3, 3, 3, 3, 6, 6, 6, 6, 6, 6, 9, 9, 9, 9, 9, 9, 12]
18 158
deque([12, 12, 12])
[0, 3, 3, 3, 3, 

([0,
  3,
  3,
  3,
  3,
  6,
  6,
  6,
  6,
  6,
  6,
  9,
  9,
  9,
  9,
  9,
  9,
  12,
  12,
  12,
  12,
  12,
  12,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  15,
  16,
  16,
  16,
  16,
  13,
  13,
  13,
  13,
  10,
  10,
  7,
  7,
  7,
  7,
  7,
  7,
  4,
  4,
  4,
  4,
  1,
  1,
  1,
  1,
  2,
  2,
  5,
  5,
  8,
  8,
  8,
  8,
  4,
  4,
  11,
  11,
  14,
  14,
  14,
  14,
  14,
  14,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  17,
  14,
  14,
  14,
  [9, 6],
  [6, 5],
  11,
  11,
  11,
  11,
  8,
  8,
  8,
  5,
  5,
  [-2, 3],
  2,
  2,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  4,
  4,
  7,
  7,
  10,
  10,
  10,
  10,
  13,
  13,
  13,
  13,
  13,
  13,
  16,
  16,
  15,
  15,
  12,
  12,
  12,
  12,
  9,
  9,
  9,
  9,
  6,
  6,
  3,
  3,
  3,
  3,
  3,
  3,
  [4, 1],
  0,
  0,
  0,
  0,
  0],
 156)

In [ ]:
# stack clear for 2 and 3 same with three steps, sliding window one step for three different

from collections import deque 
  
# sequential voting stategy
def sequential_voting(sequence = y_pred_convlstm[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1),
                      n_steps = 3):
    y_pred_new = list()
    i = 0
    for _ in range(len(sequence)):
        print(i, len(sequence))
        if i >= len(sequence)-2:
            break
        
        stack = deque([sequence[i], sequence[i+1],sequence[i+2]]) 
        print(stack)
        
        if len(set(stack)) == 1 or len(set(stack)) == 2:
            for item in set(stack):
                if stack.count(item) > 1:
                    y_pred_new.append(item) 
            while stack:
                stack.pop()
            i += n_steps
            
#         elif len(set(stack)) == 2:
#             for item in set(stack):
#                 if stack.count(item) > 1:
#                     y_pred_new.append(item)            
#             i += 1
            
        else:
            test = sc_lstm.transform([stack])
            test = test.reshape(1, 3, 1)
            out, out2 = int(np.round(model_lstm_simple.predict(test))), int(np.round(model_lstm_simple2.predict(test)))
            if out == out2:
                y_pred_new.append(out)
            else:
                y_pred_new.append([out, out2])
            print([int(np.round(model_lstm_simple.predict(test))), 
                int(np.round(model_lstm_simple2.predict(test)))])
            i += 1
            
    
        print(y_pred_new)
    return y_pred_new
    
y_pred_new = sequential_voting()
# y_pred_new, len(y_pred_new)

In [ ]:
# stack clear for 3 same with three steps, sliding window one step for 2same & three different

from collections import deque 
  
# sequential voting stategy
def sequential_voting(sequence = y_pred_convlstm[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1),
                      n_steps = 3):
    y_pred_new = list()
    i = 0
    for _ in range(len(sequence)):
        print(i, len(sequence))
        if i >= len(sequence)-2:
            break
        
        stack = deque([sequence[i], sequence[i+1],sequence[i+2]]) 
        print(stack)
        
        if len(set(stack)) == 1:
            for item in set(stack):
                if stack.count(item) > 1:
                    y_pred_new.append(item) 
            while stack:
                stack.pop()
            i += n_steps
            
        elif len(set(stack)) == 2:
            for item in set(stack):
                if stack.count(item) > 1:
                    y_pred_new.append(item)            
            i += 1
            
        else:
            test = sc_lstm.transform([stack])
            test = test.reshape(1, 3, 1)
            out, out2 = int(np.round(model_lstm_simple.predict(test))), int(np.round(model_lstm_simple2.predict(test)))
            if out == out2:
                y_pred_new.append(out)
            else:
                y_pred_new.append([out, out2])
            print([int(np.round(model_lstm_simple.predict(test))), 
                int(np.round(model_lstm_simple2.predict(test)))])
            i += 1
            
    
        print(y_pred_new)
    return y_pred_new
    
y_pred_new = sequential_voting()
# y_pred_new, len(y_pred_new)

In [ ]:
# sliding window one step for all conditions

from collections import deque 
  
# sequential voting stategy
def sequential_voting(sequence = y_pred_convlstm[y_pred_convlstm.max(axis=1) > 0.65].argmax(axis=1),
                      n_steps = 3):
    y_pred_new = list()
    i = 0
    for _ in range(len(sequence)):
        print(i, len(sequence))
        if i >= len(sequence)-2:
            break
        
        stack = deque([sequence[i], sequence[i+1],sequence[i+2]]) 
        print(stack)
        
        if len(set(stack)) == 1 or len(set(stack)) == 2:
            for item in set(stack):
                if stack.count(item) > 1:
                    y_pred_new.append(item) 
#             while stack:
#                 stack.pop()
#             i += n_steps
            i += 1
            
#         elif len(set(stack)) == 2:
#             for item in set(stack):
#                 if stack.count(item) > 1:
#                     y_pred_new.append(item)            
#             i += 1
            
        else:
            test = sc_lstm.transform([stack])
            test = test.reshape(1, 3, 1)
            out, out2 = int(np.round(model_lstm_simple.predict(test))), int(np.round(model_lstm_simple2.predict(test)))
            if out == out2:
                y_pred_new.append(out)
            else:
                y_pred_new.append([out, out2])
            print([int(np.round(model_lstm_simple.predict(test))), 
                int(np.round(model_lstm_simple2.predict(test)))])
            i += 1
            
    
        print(y_pred_new)
    return y_pred_new
    
y_pred_new = sequential_voting()
y_pred_new, len(y_pred_new)